In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e24/sample_submission.csv
/kaggle/input/playground-series-s3e24/train.csv
/kaggle/input/playground-series-s3e24/test.csv


In [36]:
df = pd.read_csv("/kaggle/input/playground-series-s3e24/train.csv", index_col = "id")

In [37]:
df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
id,,,,,,,,,,,,,,,,,,,,,
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,57,126,16.2,1,1.1,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,...,45,93,17.4,1,0.8,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,...,38,102,15.9,1,1.0,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,...,44,93,15.4,1,0.8,19,13,17,0,1


In [38]:
df.shape

(159256, 23)

I have 23 columns. 22 features, 1 binary label.

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159256 entries, 0 to 159255
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   age                  159256 non-null  int64  
 1   height(cm)           159256 non-null  int64  
 2   weight(kg)           159256 non-null  int64  
 3   waist(cm)            159256 non-null  float64
 4   eyesight(left)       159256 non-null  float64
 5   eyesight(right)      159256 non-null  float64
 6   hearing(left)        159256 non-null  int64  
 7   hearing(right)       159256 non-null  int64  
 8   systolic             159256 non-null  int64  
 9   relaxation           159256 non-null  int64  
 10  fasting blood sugar  159256 non-null  int64  
 11  Cholesterol          159256 non-null  int64  
 12  triglyceride         159256 non-null  int64  
 13  HDL                  159256 non-null  int64  
 14  LDL                  159256 non-null  int64  
 15  hemoglobin           1

All the colmns are either integer or float. I'll later change them to float

In [40]:
df.describe()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
count,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,...,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000
mean,44.306626,165.266929,67.143662,83.001990,1.005798,1.000989,1.023974,1.023421,122.503648,76.874071,...,55.852684,114.607682,14.796965,1.074233,0.892764,25.516853,26.550296,36.216004,0.197996,0.437365
std,11.842286,8.818970,12.586198,8.957937,0.402113,0.392299,0.152969,0.151238,12.729315,8.994642,...,13.964141,28.158931,1.431213,0.347856,0.179346,9.464882,17.753070,31.204643,0.398490,0.496063
min,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,44.000000,...,9.000000,1.000000,4.900000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000,0.000000
25%,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,70.000000,...,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000,0.000000
50%,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,78.000000,...,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000,0.000000
75%,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,82.000000,...,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,32.000000,44.000000,0.000000,1.000000
max,85.000000,190.000000,130.000000,127.000000,9.900000,9.900000,2.000000,2.000000,213.000000,133.000000,...,136.000000,1860.000000,21.000000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000,1.000000


BMI column (from Pranav)

In [41]:
df["BMI"] = df["weight(kg)"] / ((df["height(cm)"] / 100) * (df["height(cm)"]) / 100)
#test_df["BMI"] = test_df["weight(kg)"] / ((test_df["height(cm)"] / 100) * (test_df["height(cm)"]) / 100)

In [42]:
# TEMPORARY
df = df.iloc[:5000]

In [43]:
df["BMI"].head()

id
0    22.038567
1    23.875115
2    25.951557
3    29.320988
4    22.038567
Name: BMI, dtype: float64

## Features and Targets

In [102]:
features = df.iloc[:,:-2]
X = features.to_numpy()
X,X.shape

(array([[ 55., 165.,  60., ...,  25.,  27.,   0.],
        [ 70., 165.,  65., ...,  23.,  37.,   1.],
        [ 20., 170.,  75., ...,  31.,  53.,   0.],
        ...,
        [ 45., 170.,  70., ...,  49., 101.,   0.],
        [ 35., 175.,  80., ...,  28.,  17.,   0.],
        [ 55., 165.,  70., ...,  20.,  31.,   0.]]),
 (5000, 22))

In [103]:
df.iloc[:,-2]

id
0       1
1       0
2       1
3       0
4       1
       ..
4995    0
4996    1
4997    1
4998    0
4999    1
Name: smoking, Length: 5000, dtype: int64

In [104]:
labels = df.iloc[:,-2]
y = labels.to_numpy()
y,y.shape

(array([1, 0, 1, ..., 1, 0, 1]), (5000,))

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

## Tensor Dataset

In [48]:
from torch.utils.data import TensorDataset 

In [106]:
dataset = TensorDataset(
        torch.tensor(X).float(),
        torch.tensor(y).float()
)

In [107]:
sample = dataset[0]
print(f"INPUT: {sample[0]}\n OUTPUT: {sample[1]}")

INPUT: tensor([ 55.0000, 165.0000,  60.0000,  81.0000,   0.5000,   0.6000,   1.0000,
          1.0000, 135.0000,  87.0000,  94.0000, 172.0000, 300.0000,  40.0000,
         75.0000,  16.5000,   1.0000,   1.0000,  22.0000,  25.0000,  27.0000,
          0.0000])
 OUTPUT: 1.0


## DataLoader

In [108]:
from torch.utils.data import DataLoader 

In [109]:
dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True
)

In [110]:
for a,b in dataloader:
    print(f"IN THIS BATCH,\nINPUT: {a}\nOUTPUT: {b}")
    break

IN THIS BATCH,
INPUT: tensor([[ 35.0000, 180.0000,  95.0000,  97.0000,   1.5000,   1.5000,   1.0000,
           1.0000, 120.0000,  80.0000, 107.0000, 189.0000, 253.0000,  41.0000,
          97.0000,  15.0000,   1.0000,   1.1000,  24.0000,  37.0000,  52.0000,
           0.0000],
        [ 60.0000, 165.0000,  60.0000,  85.0000,   0.6000,   0.8000,   1.0000,
           1.0000, 148.0000,  96.0000, 111.0000, 185.0000, 121.0000,  48.0000,
         113.0000,  16.2000,   1.0000,   1.0000,  28.0000,  28.0000,  32.0000,
           0.0000],
        [ 40.0000, 155.0000,  40.0000,  66.0000,   0.6000,   0.4000,   1.0000,
           1.0000, 111.0000,  75.0000,  87.0000, 179.0000, 115.0000,  78.0000,
          75.0000,  13.0000,   1.0000,   0.6000,  15.0000,   9.0000,  10.0000,
           0.0000],
        [ 55.0000, 155.0000,  55.0000,  77.0000,   0.9000,   1.0000,   1.0000,
           1.0000, 130.0000,  90.0000,  92.0000, 225.0000, 199.0000,  52.0000,
         133.0000,  14.3000,   1.0000,   0.8000, 

In [111]:
next(iter(dataloader))

[tensor([[6.5000e+01, 1.4500e+02, 6.5000e+01, 9.2500e+01, 7.0000e-01, 2.0000e-01,
          1.0000e+00, 1.0000e+00, 1.3000e+02, 8.0000e+01, 1.1700e+02, 1.4600e+02,
          9.4000e+01, 5.3000e+01, 7.3000e+01, 1.4100e+01, 1.0000e+00, 1.0000e+00,
          1.9000e+01, 1.6000e+01, 2.5000e+01, 0.0000e+00],
         [4.0000e+01, 1.7000e+02, 7.0000e+01, 8.3000e+01, 1.2000e+00, 1.0000e+00,
          1.0000e+00, 1.0000e+00, 1.1800e+02, 7.8000e+01, 8.3000e+01, 2.0400e+02,
          5.1000e+01, 8.1000e+01, 1.1400e+02, 1.5800e+01, 1.0000e+00, 1.1000e+00,
          2.6000e+01, 1.9000e+01, 1.2500e+02, 0.0000e+00],
         [4.0000e+01, 1.8000e+02, 9.0000e+01, 9.5000e+01, 9.0000e-01, 1.2000e+00,
          1.0000e+00, 1.0000e+00, 1.2600e+02, 8.8000e+01, 8.9000e+01, 2.1900e+02,
          9.0000e+01, 5.6000e+01, 1.4500e+02, 1.6300e+01, 1.0000e+00, 8.0000e-01,
          3.3000e+01, 2.8000e+01, 1.0600e+02, 0.0000e+00],
         [6.0000e+01, 1.5000e+02, 5.0000e+01, 7.5000e+01, 9.0000e-01, 1.0000e+00,
   

We can see that each batch has 4 tensors of independent variables, and their corresponding 4 targets

## Creating a simple model without feature engineering

In [120]:
n_features = 22

model = nn.Sequential(
    nn.Linear(n_features, 128),  
    nn.ReLU(),
    nn.Linear(128, 64), 
    nn.ReLU(),
    nn.Linear(64, 1),
    nn.Sigmoid()
)

## Defining the loss

In [112]:
criterion = nn.MSELoss()
optimizer = optim.SGD( #Stocastic Gradient Descent. Adam is better
    model.parameters(),
    lr = 0.001
)

## Training Loop

In [147]:
num_epochs = 5

In [114]:
from tqdm import tqdm # For progress bar


In [137]:
device = torch.device("cuda")
# Move the model to the GPU
model.to(device)


Sequential(
  (0): Linear(in_features=22, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
  (5): Sigmoid()
)

In [ ]:
model.train()

In [153]:
for epoch in range(num_epochs):
    training_loss = 0.0
    # Initialize tqdm for the epoch - CHATGPT
    #epoch_progress = tqdm(dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    
    #for data in epoch_progress:
    for data in dataloader:
        # Setting the Gradients to zero
        optimizer.zero_grad()
        
        # Unpack data, get features and targets
        feature, target = data
        
        # Move feature and target tensors to the GPU - CHATGPT
        feature, target = feature.to(device), target.to(device)
        
        target = target.unsqueeze(1) # To make target get same shape as f.
        
        # Forward pass
        preds = model(feature)
        
        # Loss
        loss = criterion(preds, target)
        loss.backward() # Loss --> Gradients
        
        training_loss += loss.item()
        
        # Update parameters
        optimizer.step()
        
        # Update tqdm progress bar
        #epoch_progress.set_postfix(loss=loss.item())  # Update loss value in progress bar
        
    # Close tqdm progress bar for the epoch
    epoch_loss = training_loss / len(dataloader) 
    print(epoch_loss)
    
    #epoch_progress.close()


43.64
43.64
43.64
43.64
43.64


In [154]:
import torchmetrics 

# Create accuracy metric using torch metrics 
metric = torchmetrics.Accuracy(task="Binary").to(device)

for data in dataloader: 
    features, labels = data 
    features,labels = features.to(device), labels.to(device)

    outputs = model(features) 

    # Squeeze the outputs tensor to remove the extra dimension
    outputs = outputs.squeeze()

    # Assuming labels is a tensor containing probabilities or logits
    threshold = 0.5
    binary_labels = (labels >= threshold).float()  # Convert to binary values (0 or 1) based on threshold
    #rint(binary_labels)

    # Use binary_labels for calculating accuracy
    acc = metric(outputs, binary_labels)

# Calculate accuracy over the whole epoch 
acc = metric.compute() 

print(f"Accuracy on all data: {acc}") 

# Reset the metric for the next epoch (training or validation) 
metric.reset()


Accuracy on all data: 0.5636000037193298
